<a href="https://colab.research.google.com/github/shinjangwoon/jangwoonshin/blob/practice/NCAV(Net_Current_Asset_Value)(%EC%8B%A4%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Small Cap + Low-PBR

|레벨|스타일|기대CAGR|종목개수|매수전략|
|:--:|:--:|:--:|:--:|:--:|
|초,중급|밸류|20% 이상|20 ~ 30개|- 소형주(시가총액 하위 20%) <br/><br/> - PBR 하위 부터<br/>(PBR > 0.2)

## Load Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from urllib import request as rq

## Get All Stock code

In [ ]:
csv_url = '/content/drive/MyDrive/Colab Notebooks/company.csv'

company = pd.read_csv(csv_url)

In [ ]:
code_list = company['종목코드']
code_list.head(10)

In [ ]:
kospi_code = company[company['분류'] == '코스피']['종목코드']
kosdaq_code = company[company['분류'] == '코스닥']['종목코드']

##Get Market Value

In [ ]:
SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
sample_code = '035720'

In [ ]:
snap = rq.urlopen(SNAP_URL%sample_code).read()
snap_soup = BeautifulSoup(snap, 'html.parser')

In [ ]:
mv_cells = snap_soup.find('div', {'id':'svdMainGrid1'}).find_all('td', {'class':'r'})

mv = float(mv_cells[8].string.replace(',',''))
mv

##Get PBR

In [ ]:
pbr_cells = snap_soup.find('div',{'id':'corp_group2'}).find_all('dd')

float(pbr_cells[7].string.replace(',',''))

##Create Index DataFrame

In [ ]:
def filteringDf(li):
    filtering = {}

    for i in tqdm(li):
        try:
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A%s&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701' % i

            snap = rq.urlopen(SNAP_URL).read()
            snap_soup = BeautifulSoup(snap, 'html.parser')


            # get market value
            mv_cells = snap_soup.find('div',{'id':'svdMainGrid1'}).find_all('td',{'class':'r'})
            mv = float(mv_cells[8].string.replace(',','')) 

            # get pbr
            pbr_cells = snap_soup.find('div',{'id':'corp_group2'}).find_all('dd')

            pbr = float(pbr_cells[7].string.replace(',','')) 

            # get company name
            name = company[company['종목코드'] == i]['회사명'].values[0]

            filtering[name] = [i, mv, pbr]

        except (TypeError, AttributeError, IndexError, ValueError) as err : 
            pass
        
    # create DataFrame
    column_name = ['id', 'Market_Value', 'PBR']
    filtering = pd.DataFrame(filtering)
    filtering = filtering.transpose()
    filtering.columns = column_name

    return filtering


In [ ]:
# takes a long time

filtering = filteringDf(code_list[1000:1030])
filtering.head(20)

In [ ]:
# multiprocessing for parallel processing

from multiprocessing import Pool
import numpy as np

num_partitions = 12
num_cores = 4

splitted_code_list = np.array_split(code_list, num_partitions)
pool = Pool(num_cores)

filtering = pd.concat(pool.map(filteringDf, splitted_code_list))
pool.close()
pool.join()

filtering.head(20)

##Searching Stocks

In [ ]:
# PBR > 0.2 
filtering = filtering[filtering['PBR'] > 0.2]
filtering.head()

In [ ]:
filtering = filtering.sort_values(by='Market_Value')
filtering.head(20)

In [ ]:
copy_df = filtering.copy()
copy_df.head()

In [ ]:
# eliminate SPAC
copy_df = copy_df[~copy_df.index.str.contains('스팩|SPAC')]
print(copy_df.shape)
copy_df.head(20)

In [ ]:
# 20% small cap
num_quantile  = int(len(copy_df) * 0.2)

copy_df = copy_df[:num_quantile + 1]
copy_df.head(20)

In [ ]:
# Low PBR
copy_df = copy_df.sort_values(by='PBR')
copy_df = copy_df.head(30)
copy_df